In [12]:
setwd("C:/Users/DJ COMPUTERS/OneDrive/Desktop/R prog")
### Churn Prediction Model ###
### ADITYA KUMAR ROY ###

In [13]:
# Loading libraries
# Load necessary libraries
library(readr)
library(gbm)
library(dplyr)
library(car)
library(caret)
library(randomForest)
library(MLmetrics)

In [14]:
# Loading Dataset
Churn <- read_csv("Churn_Modelling.csv",show_col_types = FALSE)

In [15]:
# Exploratory Data Analysis
head(Churn)
summary(Churn)
str(Churn)
colnames(Churn)
sum(is.na(Churn))
sum(duplicated(Churn))
table(Churn$Exited)
table(Churn$Gender)
table(Churn$Geography)

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1


   RowNumber       CustomerId         Surname           CreditScore   
 Min.   :    1   Min.   :15565701   Length:10000       Min.   :350.0  
 1st Qu.: 2501   1st Qu.:15628528   Class :character   1st Qu.:584.0  
 Median : 5000   Median :15690738   Mode  :character   Median :652.0  
 Mean   : 5000   Mean   :15690941                      Mean   :650.5  
 3rd Qu.: 7500   3rd Qu.:15753234                      3rd Qu.:718.0  
 Max.   :10000   Max.   :15815690                      Max.   :850.0  
  Geography            Gender               Age            Tenure      
 Length:10000       Length:10000       Min.   :18.00   Min.   : 0.000  
 Class :character   Class :character   1st Qu.:32.00   1st Qu.: 3.000  
 Mode  :character   Mode  :character   Median :37.00   Median : 5.000  
                                       Mean   :38.92   Mean   : 5.013  
                                       3rd Qu.:44.00   3rd Qu.: 7.000  
                                       Max.   :92.00   Max.   :10.000  

spc_tbl_ [10,000 × 14] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ RowNumber      : num [1:10000] 1 2 3 4 5 6 7 8 9 10 ...
 $ CustomerId     : num [1:10000] 15634602 15647311 15619304 15701354 15737888 ...
 $ Surname        : chr [1:10000] "Hargrave" "Hill" "Onio" "Boni" ...
 $ CreditScore    : num [1:10000] 619 608 502 699 850 645 822 376 501 684 ...
 $ Geography      : chr [1:10000] "France" "Spain" "France" "France" ...
 $ Gender         : chr [1:10000] "Female" "Female" "Female" "Female" ...
 $ Age            : num [1:10000] 42 41 42 39 43 44 50 29 44 27 ...
 $ Tenure         : num [1:10000] 2 1 8 1 2 8 7 4 4 2 ...
 $ Balance        : num [1:10000] 0 83808 159661 0 125511 ...
 $ NumOfProducts  : num [1:10000] 1 1 3 2 1 2 2 4 2 1 ...
 $ HasCrCard      : num [1:10000] 1 0 1 0 1 1 1 1 0 1 ...
 $ IsActiveMember : num [1:10000] 1 1 0 0 1 0 1 0 1 1 ...
 $ EstimatedSalary: num [1:10000] 101349 112543 113932 93827 79084 ...
 $ Exited         : num [1:10000] 1 0 1 0 0 1 0 1 0 0 ...
 - attr(*

[1] "RowNumber"       "CustomerId"      "Surname"         "CreditScore"    
 [5] "Geography"       "Gender"          "Age"             "Tenure"         
 [9] "Balance"         "NumOfProducts"   "HasCrCard"       "IsActiveMember" 
[13] "EstimatedSalary" "Exited"

[1] 0

[1] 0


   0    1 
7963 2037 


Female   Male 
  4543   5457 


 France Germany   Spain 
   5014    2509    2477 

In [16]:
# Data Preprocessing

Churn_feature <- Churn |> select("CreditScore","Geography","Gender",
                                 "Age","Tenure","NumOfProducts","HasCrCard",
                                 "IsActiveMember","Exited")
Churn_feature <- as.data.frame(Churn_feature)
Churn_feature$Exited <- as.factor(Churn_feature$Exited)

In [17]:
#Splitting into train and test datasets
set.seed(123)
index <- createDataPartition(Churn_feature$Exited, p = 0.7, list = FALSE)
train <- Churn_feature[index, ]
test <- Churn_feature[-index, ]

In [18]:
# Logistic Regression Model
log_model <- glm(Exited ~ ., data = train, family = binomial)
log_pred <- predict(log_model, newdata = test, type = "response")
log_pred_class <- ifelse(log_pred > 0.7, 1, 0)
log_pred_class <- as.factor(log_pred_class)

# Evaluate Logistic Regression Model
threshold <- seq(0,1,length.out = 10)
log_pred_class <- list()
confusion_matrices <- list()
# Loop through each threshold and evaluate predictions
for(i in seq_along(threshold)){
  # Generate predictions for the current threshold
  log_pred_class_single <- ifelse(log_pred > threshold[i], 1, 0)
  
  # Ensure both log_pred_class_single and test$Exited are factors with the same levels
  log_pred_class_single <- factor(log_pred_class_single, levels = levels(test$Exited))
  test$Exited <- as.factor(test$Exited)
  
  # Calculate the confusion matrix if levels match
  if (all(levels(log_pred_class_single) == levels(test$Exited))) {
    log_cm <- confusionMatrix(log_pred_class_single, test$Exited)
    confusion_matrices[[i]] <- log_cm  # Store the result
    print(paste("Logistic Regression Model Evaluation for Threshold", threshold[i], ":"))
    print(log_cm)
  } else {
    print(paste("Levels do not match for threshold", threshold[i], ". Skipping..."))
  }
}

# Additional Metrics for Logistic Regression

# Initialize vectors to store metrics for each threshold
f1_scores <- numeric(length(threshold))
precisions <- numeric(length(threshold))
recalls <- numeric(length(threshold))

# Loop through each threshold and calculate metrics
for (i in seq_along(threshold)) {
  # Generate predictions for the current threshold
  log_pred_class_single <- ifelse(log_pred > threshold[i], 1, 0)
  
  # Ensure both log_pred_class_single and test$Exited are factors with the same levels
  log_pred_class_single <- factor(log_pred_class_single, levels = c(0, 1))
  test$Exited <- factor(test$Exited, levels = c(0, 1))
  
  # Calculate F1 Score, Precision, and Recall
  log_f1 <- F1_Score(log_pred_class_single, test$Exited, positive = "1")
  log_precision <- Precision(log_pred_class_single, test$Exited, positive = "1")
  log_recall <- Recall(log_pred_class_single, test$Exited, positive = "1")
  
  # Store metrics in vectors
  f1_scores[i] <- log_f1
  precisions[i] <- log_precision
  recalls[i] <- log_recall
}

# Combine all metrics into a data frame
metrics_df <- data.frame(
  Threshold = threshold,
  F1_Score = f1_scores,
  Precision = precisions,
  Recall = recalls
)

# View the final data frame with metrics
print(metrics_df)


[1] "Logistic Regression Model Evaluation for Threshold 0 :"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0    0    0
         1 2388  611
                                          
               Accuracy : 0.2037          
                 95% CI : (0.1894, 0.2186)
    No Information Rate : 0.7963          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.0000          
            Specificity : 1.0000          
         Pos Pred Value :    NaN          
         Neg Pred Value : 0.2037          
             Prevalence : 0.7963          
         Detection Rate : 0.0000          
   Detection Prevalence : 0.0000          
      Balanced Accuracy : 0.5000          
                                          
      

In [19]:
# 2. Random Forest Model
# Random Forest Model with Hyperparameter Tuning
control <- trainControl(method = "cv", number = 5)  # 5-fold cross-validation
tune_grid <- expand.grid(mtry = c(2, 3, 4))  # Testing different mtry values

rf_tuned <- train(Exited ~ ., data = train, method = "rf", tuneGrid = tune_grid,
                  trControl = control, ntree = 100)
print(rf_tuned)

# Predict and evaluate using the tuned Random Forest model
rf_pred_tuned <- predict(rf_tuned, newdata = test)
rf_cm_tuned <- confusionMatrix(rf_pred_tuned, test$Exited)
print("Tuned Random Forest Model Evaluation:")
print(rf_cm_tuned)

# Variable Importance for Random Forest
print("Random Forest Variable Importance:")
print(varImp(rf_tuned))

# Additional Metrics for Random Forest
rf_f1 <- F1_Score(rf_pred_tuned, test$Exited, positive = "1")
rf_precision <- Precision(rf_pred_tuned, test$Exited, positive = "1")
rf_recall <- Recall(rf_pred_tuned, test$Exited, positive = "1")

cat("Random Forest - F1 Score:", rf_f1, "\n")
cat("Random Forest - Precision:", rf_precision, "\n")
cat("Random Forest - Recall:", rf_recall, "\n")

Random Forest 

7001 samples
   8 predictor
   2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 5601, 5601, 5601, 5601, 5600 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.8534491  0.4344852
  3     0.8547350  0.4661610
  4     0.8475929  0.4525314

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 3.
[1] "Tuned Random Forest Model Evaluation:"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 2302  334
         1   86  277
                                         
               Accuracy : 0.86           
                 95% CI : (0.847, 0.8722)
    No Information Rate : 0.7963         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.4916         
                                         
 Mcnemar's Test P-Value : < 2.2e-16      

In [20]:
### Conclusion ###
## Random Forest Model performs better.
## Logistic Model is also efficient at 0.55